In [495]:
# separate hg38 by chromosome
hg38_ref_path = '/autofs/bal20/ykzhou/ref/hg38.no_alt.fa'
with open(hg38_ref_path, 'r') as file:
    hg38_ref = file.readlines()
for i in range(len(hg38_ref)):
    if hg38_ref[i][0] == '>':
        chr_name = hg38_ref[i][1:-1]
        f = open('/autofs/bal20/ykzhou/ref/hg38_noalt_chr/' + chr_name + '.fa', 'x')
        f.write(hg38_ref[i])
    else:
        f.write(hg38_ref[i])

# group reads info at each info
flag_ps = [[chrom[ch][r][8][:] for r in range(len(chrom[ch])) if chrom[ch][r][26] != ''] for ch in range(24)]
for ch in range(24):
    print(collections.Counter(flag_ps[ch]))

In [22]:
import collections
import os
import shutil

In [23]:
chrom = []
chrom_list = []
for i in range(1, 23):
    chrom_list.append(str(i))
chrom_list.append('X')
chrom_list.append('Y')

tag = ['s2:i:', 'de:f:', 'zd:i:', 'SA:Z:', 'MD:Z:', 'rl:i:', 'HP:i:', 'PC:i:', 'PS:i:']

for ch in range(len(chrom_list)):
    with open('/autofs/bal20/ykzhou/HG00733_ONT_PanGenome/phase_8x_hg38/haplotag_seq/chr' + chrom_list[ch] + '.sam', 'r') as file:
        chr_raw = file.readlines()
    chr_raw = [s.strip() for s in chr_raw]
    chri = [['' for c in range(27)] for r in range(len(chr_raw))]
    for r in range(len(chr_raw)):
        read_str = chr_raw[r].split()
        for i in range(18):
            chri[r][i] = read_str[i]
        for i in range(9):
            ls = [s for s in read_str[18:] if tag[i] in s]
            if ls:
                chri[r][i+18] = ls[0]
    chrom.append(chri)

PSed reads: half flaged 16, half 0, all tp:A:P
<10% SA
all: 0 16 256 272 2048 2064
all: * 0 0

s1 s2 de zd SA MD rl HP PC PS
17 18 19 20 21 22 23 24 25 26

PS No: 252 261 214 232 241 193 167 151 182 172 133 136 96 106 111 77 86 99 52 95 61 65 220 7 3409

In [18]:
chrom_list = []
for i in range(1, 23):
    chrom_list.append(str(i))
chrom_list.append('X')
chrom_list.append('Y')

tag = ['s2:i:', 'de:f:', 'zd:i:', 'SA:Z:', 'MD:Z:', 'rl:i:', 'HP:i:', 'PC:i:', 'PS:i:']

root = '/autofs/bal20/ykzhou/HG00733_ONT_PanGenome/phase_8x_hg38/'
header_file = root + 'haplotag_seq/header.sam'
vcfs = root + 'tmp_svcalling/vcf_nogt_sr1_1/'
tmp_sam = vcfs + 'tmp.sam'
tmp_bam = vcfs + 'tmp.bam'

for ch in range(24):
    with open('/autofs/bal20/ykzhou/HG00733_ONT_PanGenome/phase_8x_hg38/haplotag_seq/chr' + chrom_list[ch] + '.sam', 'r') as file:
        chr_raw = file.readlines()
    file.close()
    chr_raw = [s.strip() for s in chr_raw]
    chri = [['' for c in range(27)] for r in range(len(chr_raw))]
    for r in range(len(chr_raw)):
        read_str = chr_raw[r].split()
        for i in range(18):
            chri[r][i] = read_str[i]
        for i in range(9):
            ls = [s for s in read_str[18:] if tag[i] in s]
            if ls:
                chri[r][i+18] = ls[0]

    psset = set([chri[i][26] for i in range(len(chri))])
    psset.remove('')
    pslist = list(psset)
    ps_int = [int(pslist[ps][5:]) for ps in range(len(pslist))]
    ps_int.sort()
    pslist_sort = ['PS:i:' + str(ps_int[i]) for i in range(len(ps_int))]

    for ps_idx in range(len(pslist_sort)):
        ps = pslist_sort[ps_idx] # ps = 'PS:i:...'
        read_ps = [chri[i] for i in range(len(chri)) if chri[i][26] == ps]
        hp1 = [read_ps[i][:11] for i in range(len(read_ps)) if read_ps[i][24] == 'HP:i:1']
        hp2 = [read_ps[i][:11] for i in range(len(read_ps)) if read_ps[i][24] == 'HP:i:2']
    
        shutil.copyfile(header_file, tmp_sam) # create header
        sam_file = open(tmp_sam, 'a')
        for r in range(len(hp1)):
            read_str = '\t'.join(hp1[r]) + '\n'
            sam_file.write(read_str)
        sam_file.close()
        os.system('samtools view -@40 -b ' + tmp_sam + ' | samtools sort  -@40 > ' + tmp_bam)
        os.system('samtools index -@40 ' + tmp_bam)
        os.system('cuteSV ' + tmp_bam + ' /autofs/bal20/ykzhou/ref/hg38.no_alt.fa ' + vcfs + 'chr' + chrom_list[ch] + '_ps' + pslist_sort[ps_idx][5:] + '_hp1' + \
            '.vcf ' + vcfs + ' -s 1 --max_cluster_bias_INS 100 --diff_ratio_merging_INS 0.3 ' + \
            '--max_cluster_bias_DEL 100 --diff_ratio_merging_DEL 0.3 -t 40 --report_readid')
        
        shutil.copyfile(header_file, tmp_sam) # create header
        sam_file = open(tmp_sam, 'a')
        for r in range(len(hp2)):
            read_str = '\t'.join(hp2[r]) + '\n'
            sam_file.write(read_str)
        sam_file.close()
        os.system('samtools view -@40 -b ' + tmp_sam + ' | samtools sort  -@40 > ' + tmp_bam)
        os.system('samtools index -@40 ' + tmp_bam)
        os.system('cuteSV ' + tmp_bam + ' /autofs/bal20/ykzhou/ref/hg38.no_alt.fa ' + vcfs + 'chr' + chrom_list[ch] + '_ps' + pslist_sort[ps_idx][5:] + '_hp2' + \
            '.vcf ' + vcfs + ' -s 1 --max_cluster_bias_INS 100 --diff_ratio_merging_INS 0.3 ' + \
            '--max_cluster_bias_DEL 100 --diff_ratio_merging_DEL 0.3 -t 40 --report_readid')